<a href="https://colab.research.google.com/github/githubvishwas/lasso_cae_ml/blob/main/Exercise-1_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
pip install lasso-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 23.8 MB 1.4 MB/s 
     |████████████████████████████████| 237 kB 70.7 MB/s 
     |████████████████████████████████| 51 kB 7.3 MB/s 
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1310 sha256=244849d443a22b796f711b677d6a5e13c4aed291f2df079103d78e0ba7fe957b
  Stored in directory: /root/.cache/pip/wheels/46/ef/c3/157e41f5ee1372d1be90b09f74f82b10e391eaacca8f22d33e
Successfully built sklearn


In [4]:
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import numpy as np
import glob
import ntpath
from ipywidgets import Image, Layout
from lasso.dyna import D3plot, ArrayType
from sklearn.decomposition import PCA
from sklearn.manifold import LocallyLinearEmbedding, MDS

In [9]:
import os
from google.colab import drive
MOUNTPOINT = '/content/gdrive'
DATADIR = os.path.join(MOUNTPOINT, 'My Drive', 'lasso_python_ml')
drive.mount(MOUNTPOINT)

Mounted at /content/gdrive


In [16]:
#filepath_list = glob.glob("exercise-1/d3plots/*.d3plot")
filepath_list = glob.glob(DATADIR + "/exercise-1/d3plots/*.d3plot")
print(f"Found {len(filepath_list)} files.")

Found 100 files.


In [17]:
# we select the last timestep
last_timestep = -1

# let's collect node displacement from all d3plots
fields = []

for i_filepath, filepath in enumerate(filepath_list):
    
    # print progress
    print(f"filepath {filepath} ({i_filepath + 1}/{len(filepath_list)})", end='\r')
    
    # load d3plot
    d3plot = D3plot(filepath)
    
    # extract node displacement at last timestep
    node_displacement = d3plot.arrays[ArrayType.node_displacement]
    fields.append(node_displacement[last_timestep])

# print end
print(f"Done ({i_filepath + 1}/{len(filepath_list)})")

Done (100/100)


In [18]:
# convert list into a numpy array
data = np.stack(fields)

print("(n_simulations, n_nodes, xyz)")
print(data.shape)

(n_simulations, n_nodes, xyz)
(100, 2500, 3)


In [19]:
# let's select x-displacement only
#data_x = data[:, :, 0]
data_x = data.reshape(100, -1)
data_x.shape

(100, 7500)

In [20]:
# run PCA on x-displacement
model = PCA(n_components=3)
model.fit(data_x)

#model = LocallyLinearEmbedding(n_components=3)
#model.fit(data_x)

#model = MDS(n_components=3)
#model.fit(data_x)

# run SVD
#u, s, v = np.linalg.svd(x_data)

PCA(n_components=3)

In [21]:
# displace singular values/eigenvalues
fig = go.FigureWidget()
fig.add_trace(
    go.Scatter(x=np.arange(len(model.singular_values_)), 
               y=model.singular_values_, 
               mode='lines+markers',
               name="Singular Values"
              )
)

fig.update_layout(
    title='Singular Values of PCA',
    xaxis_title='Eigenvalue Index',
    yaxis_title='Eigenvalue',
)

fig.show()

In [22]:
# use only most important values
n_modes = 3

# transform the data into reduced space
data_x_transformed = model.transform(data_x)[:, :n_modes]
#data_x_transformed = model.embedding_

In [25]:
# load images
image_data = {}
for img_filepath in glob.glob(DATADIR + '/exercise-1/images/*.png'):
    with open(img_filepath, "rb") as f:
        print(img_filepath, end='\r')
        number = ntpath.basename(img_filepath).split('.')[0]
        image_data[int(number)] = f.read()


In [27]:
# create basic graph
fig = go.FigureWidget(
    data=[
        dict(
            type='scatter3d',
            x=data_x_transformed[:, 0],
            y=data_x_transformed[:, 1],
            z=data_x_transformed[:, 2],
            mode='markers',
            name="Embedding Simulation",
            marker={
               "size": 5
            },
            text=filepath_list,
            hoverinfo="text",
            opacity=0.99
        )
    ]
)

# create image widget
img_scale = 1.2
width = 500 * img_scale 
height = 375 * img_scale

image_widget = Image(
    value=image_data[0],
    layout=Layout(height=f'{height}px', width=f'{width}px')
)

# select widget
scatter = fig.data[0]

# define hover function for scatter
# to show images on hover
def hover_fn(trace, points, state):

    ind = points.point_inds[0]
    filepath = filepath_list[ind]
    name = ntpath.basename(filepath).split('.')[0].replace("run","")
    number = int(name)

    # Update image widget
    image_widget.value = image_data[number]

scatter.on_hover(hover_fn)

# some styling 
fig.update_layout(title='3D Embedding of Simulations',
                  margin=go.layout.Margin(
                    l=0, #left margin
                    r=0, #right margin
                    b=0, #bottom margin
                    t=0  #top margin
                  ),
                  hovermode="closest",
                 )

#import inspect
#print([e[0] for e in inspect.getmembers(image_widget)])

from ipywidgets import HBox, VBox
HBox([fig,
      image_widget])

#fig
fig.show()
#image_widget